In [ ]:
import geopandas as gpd

### Make polygon
This notebook takes a shape in GeoPackage format or similar and generates an outline in `poly` format that can be used by `osmosis` to extract that area.

In [ ]:
# Manage inputs and outputs
study_area_path = "../../resources/spatial/study_area.gpkg"
output_path = "../../results/network/study_area.poly"

if "snakemake" in locals():
    study_area_path = snakemake.input[0]
    output_path = snakemake.output[0]

In [8]:
# Convert zones to POLY format
df = gpd.read_file(study_area_path)
df = df.to_crs("EPSG:4326")

df["aggregate"] = 0
area = df.dissolve(by = "aggregate")["geometry"].values[0]

if not hasattr(area, "exterior"):
    print("Selected area is not connected -> Using convex hull.")
    area = area.convex_hull

data = []
data.append("polyfile")
data.append("polygon")

for coordinate in area.exterior.coords:
    data.append("    %e    %e" % coordinate)

data.append("END")
data.append("END")

with open(output_path, "w+") as f:
    f.write("\n".join(data))